# Imports

In [ ]:
import sys
sys.path.append('../')
from datetime import datetime
import numpy as np
import pandas as pd

# import src.settings as settings
# from src.preprocess.preprocess import get_and_combine_cbs_tables, rename_and_subset_cols, get_region_period_spec_val_subtable
from run_all.main_preprocess import load_data, add_features
from utilities.utilities import get_latest_file

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Preprocess

In [ ]:
%%time
# Load original sources and combine to one DataFrame
df_dataset_WMO = load_data(region='gemeente')

In [ ]:
df_dataset_WMO.shape

In [ ]:
datapath = '../../data/'
filename = 'df_WMO_WIJK_HOUSEHOLDS_POP_LEVY_absolute_gemeente.parquet.gzip'

df_dataset_WMO.to_parquet(datapath + filename,
              compression='gzip')

# df_dataset_WMO = pd.read_parquet(datapath + filename)

In [ ]:
# 21:44
df_dataset_WMO.head()

In [ ]:
df_dataset_WMO.shape

In [ ]:
list(df_dataset_WMO.columns)

In [ ]:
drop_list = ['popcodea',
 'popcodeb',
 'popcodec',
 'popcoded',
 'popcodee',
 'popcodef',
 'popcodeg',
 'popcodeh',
 'popcodei',
 'popcodej',
 'popcodek',
 'popcodel',
 'popcodem',
 'popcoden',
 'popcodeo',
 'popcodep',
 'popcodeq',
 'popcoder',
 'popnaama',
 'popnaamb',
 'popnaamc',
 'popnaamd',
 'popnaame',
 'popnaamf',
 'popnaamg',
 'popnaamh',
 'popnaami',
 'popnaamj',
 'popnaamk',
 'popnaaml',
 'popnaamm',
 'popnaamn',
 'popnaamo',
 'popnaamp',
 'popnaamq',
 'popnaamr',
'popkoppelvariabeleregiocode']

In [ ]:
df_dataset_WMO = df_dataset_WMO.drop(drop_list, axis=1)

In [ ]:
df_dataset_WMO.head()

In [ ]:
df_dataset_WMO.fillna(value=np.nan, inplace=True)

In [ ]:
%%time
# Feature engineering to get more features
df_dataset_WMO_with_features = add_features(df_dataset_WMO)

In [ ]:
df_dataset_WMO_with_features.shape

In [ ]:
datapath = '../../data/'
filename = 'df_WMO_WIJK_HOUSEHOLDS_POP_LEVY_features.parquet.gzip'

df_dataset_WMO_with_features.to_parquet(datapath + filename,
              compression='gzip')

# df_dataset_WMO = pd.read_parquet(datapath + filename)

## Optional: Write temporary result

In [ ]:
# suffix_datetime = datetime.strftime(datetime.now(), format='%Y%m%d%H%M')

# df_dataset_WMO_with_features.to_parquet(f'../../data/df_preprocess_WMO_{suffix_datetime}.parquet.gzip',
#               compression='gzip')

# Train

## Optional: Load previous dataset

In [ ]:
# ## HARDCODED
# datapath = '../../data/'
# filename = 'df_preprocess_WMO_202103211137.parquet.gzip'
# df = pd.read_parquet(datapath + filename)

# ## SELECT LAST FILE
# datapath = '../../data/'
# df = get_latest_file(filename_str_contains='df_preprocess_WMO_', datapath=datapath, filetype='parquet')

## Train model
Code will be added later